# **Grammatical Error Detection Tool using RNNs**

# Adding Libraries

In [0]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.preprocessing import MinMaxScaler
print(tf.__version__)
import sys
print(sys.version)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

%matplotlib inline

2.2.0
3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


Adding Libraries to Google Colab

In [0]:
from google.colab import files
file = files.upload()

Saving in_domain_dev.csv to in_domain_dev.csv


In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/Datasets")

# Importing Data from the two soucres and concatenating them


In [0]:
data = pd.read_csv("in_domain_train.csv")
data_dev = pd.read_csv("in_domain_dev.csv")
data = data.values
data_dev = data_dev.values
x_train = data[:,3]
y_train = data[:,1]

x_dev = data_dev[:,3]
y_dev = data_dev[:,1]

y = np.concatenate((y_dev,y_train))
x = np.concatenate((x_dev,x_train))
import random
random.shuffle(y)
random.shuffle(x)  
y_t_0 = y[1500:]
y_d_0 = y[:1500]
x_t_0 = x[1500:]
x_d_0 = x[:1500]

In [0]:
f = open('entries_train - Copy.txt')
lines=f.readlines()
sentences = []
y = []
line = 0
cur = 0
for x in lines:
    cur = cur +1
    if x=='\n' : continue
    else: line=line+1
    if line>=60000: 
        print(x,cur)
        break
    sentences.append(x.split('\t')[4])
    if x.split('\t')[0] == '0':
        y.append(1)
    else:
        y.append(0)
f.close()

0	307365	http://lang-8.com/18840/journals/84655	21	They asked me to explain `` Ionic Bond `` in English .
 65781


In [0]:
print(len(y),len(sentences))

59999 59999


In [0]:
zeros = 0
ones = 0
for l in y:
    if l==0:
        zeros = zeros+1
    else:
        ones = ones+1
print(zeros,ones)

29389 30610


In [0]:
y_t_1 = y[5000:]
y_d_1 = y[:5000]
x_t_1 = sentences[5000:]
x_d_1 = sentences[:5000]

y_t = np.concatenate((y_t_1,y_t_0))
y_d = np.concatenate((y_d_1,y_d_0))
sentences_t = np.concatenate((x_t_1,x_t_0))
sentences_d = np.concatenate((x_d_1,x_d_0))

print(len(y_t),len(y_d),sentences_t[:3],sentences_d[:3])

62577 6500 ['I really miss them .\n' "I 'm not sure how I spend more 3 weeks here ."
 'Oh my god !\n'] ['Good luck on your new start !\n'
 'My teacher is going to move to change his job .\n'
 'He is a so nice guy and taught me English very kindly and was willing to accept my getting off the track .\n']


# Text Processing

In [0]:
embedding_dim = 100
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences_t)

vocab_size = len(tokenizer.word_index)+1
sequences = tokenizer.texts_to_sequences(sentences_t)
padded = pad_sequences(sequences,maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(sentences_d)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

y_train_np = np.array(y_t,np.float32)
y_dev_np = np.array(y_d,np.float32)
testing_padded_np = np.array(testing_padded,np.float32)
padded_np = np.array(padded,np.float32)

y_train_tf = tf.convert_to_tensor(y_train_np, np.float32)
y_dev_tf = tf.convert_to_tensor(y_dev_np, np.float32)
testing_padded_tf = tf.convert_to_tensor(testing_padded_np, np.float32)
padded_tf = tf.convert_to_tensor(padded_np, np.float32)

In [0]:
print(padded_np)

# Adding Pre-trained GLOVE Word Embeddings

In [0]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.array(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
count = 0
for word,i in tokenizer.word_index.items():
    cur = cur+1
print(cur)

88791


In [0]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

# Building the RNN

In [0]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    tf.keras.layers.LSTM(64,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss = 'binary_crossentropy', optimizer=opt,metrics = ['accuracy'])
print(model.summary())
print(y_dev_tf.shape,y_train_tf.shape,padded_tf.shape,testing_padded_tf.shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 100)          2301100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 2,343,927
Trainable params: 42,699
Non-trainable params: 2,301,228
_______________________________________

Checking the shapes before training the model

In [0]:
print(testing_padded_tf.shape,y_dev_tf.shape,padded_tf.shape,y_train_tf.shape)

(6500, 120) (6500,) (62577, 120) (62577,)


# Training the model

In [116]:
num_epochs = 5
model.fit(padded_tf, y_train_tf, epochs=num_epochs, validation_data=(testing_padded_tf, y_dev_tf))

Epoch 1/5
1956/1956 [==============================] - 142s 73ms/step - loss: 0.5849 - accuracy: 0.6764 - val_loss: 0.6273 - val_accuracy: 0.6422
Epoch 2/5
1956/1956 [==============================] - 141s 72ms/step - loss: 0.5818 - accuracy: 0.6815 - val_loss: 0.6267 - val_accuracy: 0.6414
Epoch 3/5
1956/1956 [==============================] - 135s 69ms/step - loss: 0.5812 - accuracy: 0.6815 - val_loss: 0.6376 - val_accuracy: 0.6352
Epoch 4/5
1956/1956 [==============================] - 131s 67ms/step - loss: 0.5808 - accuracy: 0.6819 - val_loss: 0.6287 - val_accuracy: 0.6340
Epoch 5/5
1956/1956 [==============================] - 133s 68ms/step - loss: 0.5802 - accuracy: 0.6830 - val_loss: 0.6285 - val_accuracy: 0.6406


# Testing the model on custom text

In [115]:
custom = input("Sentence: ")
custom_test_1 = np.array([custom])
custom_1 = tokenizer.texts_to_sequences(custom_test_1)
flag = 0
for q in custom_1[0]:
    if q!=1: 
        flag = 1
        break
if flag == 0: 
    print("All words out of vocabulary!")
custom_2 = pad_sequences(custom_1,maxlen=max_length)
custom_np = np.array(custom_2,np.float32)
custom_tf = tf.convert_to_tensor(custom_np,np.float32)
prob = model.predict(custom_tf)
if prob<0.65:
    print("Grammatically \033[1m INCORRECT \033[0m with probability: ",1-prob)
else:
    print("Grammatically \033[1m CORRECT \033[0m with probability: ",prob)
print(prob)
print(custom_test_1)

Sentence: This is an wrong sentences.
Grammatically  INCORRECT  with probability:  [[0.3591038]]
[[0.6408962]]
['This is an wrong sentences.']
